In [1]:
import numpy as np

In [2]:
class Tensor:
    """
    Tensor
    """
    def __init__(self, numpy_array):
        if not isinstance(numpy_array, np.ndarray):
            raise ValueError("The class must be initialized with a Numpy array")
        self.value = numpy_array
        self.previous_tensors = None
        self.operation = None  # type of operation done to create this tensor.
        self.grad = np.full(self.value.shape, 1)
        
    def backward(self):
        self._compute_grads(self)
    
    def _compute_grads(self, tensor):
        if tensor.previous_tensors is None:
            print('stopped')
#             print('tensor:', tensor)
            return
        else:
            t1, t2 = tensor.previous_tensors
#             print('t1_computed:', str(t1_computed))
#             print('t2_computed:', str(t2_computed))
#             print('t1:', str(t1))
#             print('t2:', str(t2))
#             print('tensor.grad:', str(tensor.grad))
#             if t1_computed is None or t2_computed is None:
#                 return
            if tensor.operation == 'add':
                print('r add')
                t1.grad = tensor.grad
                t2.grad = tensor.grad
            elif tensor.operation == 'mul':
                print('***1')
                print('r mull')
                print('t1.value:', t1.value)
                print('t2.value:', t2.value)
                print('tensor.value', tensor.value)
                print('tensor.grad:', tensor.grad)
                t1.grad = tensor.grad * t2.value # * t1.grad
                t2.grad = tensor.grad * t1.value # * t2.grad
                print('t1.grad:', t1.grad)
                print('t2.grad:', t2.grad)
                print('***2')
            else:
                raise ValueError("Invalid operation")
            tensor.grad = None  # remove for saving memory
            self._compute_grads(t1)
            self._compute_grads(t2)
    
            
    def __add__(self, other):
        if isinstance(other, Tensor):
            # check shapes
            if not self.value.shape == other.value.shape:
                raise ValueError("Tensors must have the same shape. shape1: {}, shape2: {}".\
                                     format(str(self.value.shape), str(other.value.shape)))
            operation_result = self.value + other.value
            new_tensor = Tensor(operation_result)
            new_tensor.operation = 'add'
            new_tensor.previous_tensors = [self, other]
            print('add operation happened')
            return new_tensor
        else:
            raise ValueError("Operands must be of `Tensor` type.")
            
    
    def __mul__(self, other):
        if isinstance(other, Tensor):
            # check shapes
            if not self.value.shape == other.value.shape:
                raise ValueError("Tensors must have the same shape. shape1: {}, shape2: {}".\
                                     format(str(self.value.shape), str(other.value.shape)))
            operation_result = self.value * other.value
            new_tensor = Tensor(operation_result)
            new_tensor.operation = 'mul'
            new_tensor.previous_tensors = [self, other]
            print('mul operation happened')
            return new_tensor
        else:
            raise ValueError("Operands must be of `Tensor` type.")
    
    def __str__(self):
        return self.value.__repr__()

In [3]:
arr1 = np.random.rand(2, 3)
arr2 = np.random.rand(2, 3)

In [4]:
t1 = Tensor(arr1)
t2 = Tensor(arr2)
print('t1:', t1)
print('t2:', t2)

t1: array([[0.72513608, 0.43521203, 0.79568601],
       [0.44408007, 0.88073825, 0.48013565]])
t2: array([[0.92854234, 0.37359521, 0.05403239],
       [0.30128   , 0.60212432, 0.92970222]])


In [5]:
t3 = t1 * Tensor(np.full(arr1.shape, 5)) * Tensor(np.full(arr1.shape, 5)) + t2 * Tensor(np.full(arr1.shape, 3))
t4 = (t3 + Tensor(np.full(arr1.shape, 2))) * Tensor(np.full(arr1.shape, 7))

mul operation happened
mul operation happened
mul operation happened
add operation happened
add operation happened
mul operation happened


In [6]:
t4.backward()

***1
r mull
t1.value: [[22.91402912 14.00108647 22.05424729]
 [14.00584174 25.82482927 16.79249792]]
t2.value: [[7 7 7]
 [7 7 7]]
tensor.value [[160.39820381  98.00760528 154.37973104]
 [ 98.0408922  180.77380487 117.54748546]]
tensor.grad: [[1 1 1]
 [1 1 1]]
t1.grad: [[7 7 7]
 [7 7 7]]
t2.grad: [[22.91402912 14.00108647 22.05424729]
 [14.00584174 25.82482927 16.79249792]]
***2
r add
r add
***1
r mull
t1.value: [[3.62568042 2.17606017 3.97843003]
 [2.22040035 4.40369126 2.40067825]]
t2.value: [[5 5 5]
 [5 5 5]]
tensor.value [[18.12840209 10.88030084 19.89215013]
 [11.10200176 22.01845631 12.00339125]]
tensor.grad: [[7 7 7]
 [7 7 7]]
t1.grad: [[35 35 35]
 [35 35 35]]
t2.grad: [[25.37976293 15.23242118 27.84901019]
 [15.54280246 30.82583883 16.80474775]]
***2
***1
r mull
t1.value: [[0.72513608 0.43521203 0.79568601]
 [0.44408007 0.88073825 0.48013565]]
t2.value: [[5 5 5]
 [5 5 5]]
tensor.value [[3.62568042 2.17606017 3.97843003]
 [2.22040035 4.40369126 2.40067825]]
tensor.grad: [[35 35 3

In [7]:
t1.grad

array([[175, 175, 175],
       [175, 175, 175]])

In [8]:
t2.grad

array([[21, 21, 21],
       [21, 21, 21]])

In [9]:
t3.grad